In [ ]:
#импортирование библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from plotly import graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from scipy import stats as st
import datetime
import math as mth

import warnings
warnings.filterwarnings("ignore")

#pd опции 
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)
sns.set()

In [ ]:
# прочитаем файл
file_name = 'test_data.xlsx'
task = pd.read_excel(file_name, sheet_name = 'task')

In [ ]:
display(task.head())

In [ ]:
call = pd.read_excel(file_name, sheet_name = 'call')
display(call.head())

In [ ]:
result_product = pd.read_excel(file_name, sheet_name = 'result_product')
display(result_product.head())

In [ ]:
dict_calls = pd.read_excel(file_name, sheet_name = 'Словари', usecols = 'A:B', header = 1)
display(dict_calls.head())

dict_tasks = pd.read_excel(file_name, sheet_name = 'Словари', usecols = 'D:E',  header = 1)
display(dict_tasks.head())

dict_base = pd.read_excel(file_name, sheet_name = 'Словари', usecols = 'G:H',  header = 1)
display(dict_base.head())

# 1
1. Каждая запись в task это задание совершить коммуникацию с потенциальным клиентом для продажи продукта (т.е. task=клиент).
2. В рамках задания (task) может быть несколько звонков.
3. При успешной продаже (результат задания = Успешно) продукта можно проследить дальнейшие действия клиента (result_product)

## Задача 
Составить общую таблицу с данными из всех источников и словарей.

In [ ]:
# объединить таблицы в одну

full_table = call.merge(task, on = 'task_id', how = 'left').merge(result_product, on = 'hid', how = 'left')
display(full_table.head(5))

In [ ]:
#full_table.isna().sum()
full_table.info()

# 2

## Задача

1. На основе данных из таблицы, созданной по Заданию 1, необходимо расчитать общую воронку продаж за все время наблюдений по базам.
2. Расчитать показатели сотрудников, количество звонков , продажную конверсию ("Дозвон, Успешно"/("Дозвон, Отказ" + "Дозвон, успешно")), среднее время в звонке.
3. На основе решения П.1 визуализировать полученные данные в виде графиков/диаграмм, сформировать выводы и предложения. 


In [ ]:
# воронка событий 
print('Всего заданий = ', full_table['task_id'].nunique())
#display(task['task_result_id'].value_counts())

#display(full_table.pivot_table)
print('Успешных заданий = ', task.query('task_result_id == 6')['task_id'].count())
print('Одобренных продуктов = ', result_product[~result_product['approve_dt'].isna()]['hid'].count())
print('Использованных продуктов = ', result_product[~result_product['using_dt'].isna()]['hid'].count())
d = {'col1': [1, 2], 'col2': [3, 4]}
sales_funnel = pd.DataFrame(data = {'event': ['Всего заданий', 
                                              'Успешных заданий', 
                                              'Одобренных продуктов',
                                              'Использованных продуктов'
                                             ],
                                    'n_tasks': [full_table['task_id'].nunique(), 
                                                task.query('task_result_id == 6')['task_id'].count(),
                                               result_product[~result_product['approve_dt'].isna()]['hid'].count(),
                                               result_product[~result_product['using_dt'].isna()]['hid'].count()]
                                               
                                   })
display(sales_funnel)
fig = go.Figure(go.Funnel(
    y = sales_funnel['event'],
    x = sales_funnel['n_tasks'],
    textinfo = "value+percent initial"))
fig.update_layout(title = "Воронка событий")
fig.show()

display(sales_funnel)
fig = go.Figure(go.Funnel(
    y = sales_funnel['event'],
    x = sales_funnel['n_tasks'],
    textinfo = "value+percent previous"))
fig.update_layout(title = "Воронка событий")
fig.show()

In [ ]:
# Расчитать показатели сотрудников, 
# количество звонков , среднее время в звонке.

agents_data = full_table.groupby('agent_name').agg(
                                n_calls = ('call_id','count'),
                                avg_call_sec = ('call_time_sec', 'mean')
                                    ).reset_index()
#display(agents_data)

#продажную конверсию ("Дозвон, Успешно"/("Дозвон, Отказ" + "Дозвон, успешно")), 
agents_tmp = full_table.pivot_table(values = 'call_id', 
                       index = 'agent_name', 
                       columns =  'call_result_id',
                       aggfunc = 'count').reset_index()
print(agents_tmp.columns)
agents_tmp  = agents_tmp.rename(columns = {1:'call_reject', 3:"call_success"})

agents_tmp['conversion'] = agents_tmp['call_success']/(agents_tmp['call_reject']+agents_tmp['call_success'])
display(agents_tmp.head())

agents_data = agents_data.merge(agents_tmp[['agent_name','call_success', 'call_reject', 'conversion']],
                                on = 'agent_name', 
                                how = 'left')
agents_data['conversion'] = agents_data['conversion'].fillna(0)
display(agents_data.head())

In [ ]:
# графики
# количество звонков

plt.figure(figsize = (10,6))
ax = sns.distplot(agents_data['n_calls'], kde = False)
ax.set(title = "Гистограмма признака =  Количество звонков от оператора",
        xlabel = 'Количество звонков',
        ylabel = "Частота")
plt.show() 

# средняя продолжительность звонков 
plt.figure(figsize = (10,6))
ax = sns.distplot(agents_data['avg_call_sec'], kde = False)
ax.set(title = "Гистограмма признака =  Средняя продолжительность звонка",
        xlabel = 'Средняя продолжительность звонка, сек',
        ylabel = "Частота")
plt.show() 

# продажная конверсия
plt.figure(figsize = (10,6))
ax = sns.distplot(agents_data['conversion'],kde = False)
ax.set(title = "Гистограмма признака =  Продажная конверсия",
        xlabel = 'Продажная конверсия',
        ylabel = "Частота")
plt.show() 

In [ ]:
agents_data.query('n_calls > 1500')

In [ ]:
agents_data.nlargest(10,'avg_call_sec')

In [ ]:
print(agents_data['conversion'].describe())
agents_data['conv_level'] = pd.qcut(agents_data['conversion'], 
                                        q = 3, 
                                        labels=['ниже среднего', 'средняя','выше среднего'])
display(agents_data.head())

ax = sns.relplot(x='avg_call_sec', 
          y = 'n_calls',
          hue="conv_level", style="conv_level",
          data = agents_data,
          height=8, # make the plot 5 units high
          aspect=1) # height should be three times width)
#ax.set(xlim=(0,600),ylim=(0,2000))
ax.set(title = 'Распределение операторов по продолжительности и количеству с звонков для разных уровней продажной конверсии',
                xlabel = 'Средняя продолжительность звонка, сек',
                ylabel = 'Количество звонков')
plt.show()